In [1]:
print("Status: Working")

Status: Working


In [2]:
import os
import wandb
from dotenv import load_dotenv
load_dotenv()

WNB_API_KEY = os.getenv("WNB_API_TOKEN")
if WNB_API_KEY is None:
    raise EnvironmentError(f"Environment variable {WNB_API_KEY} is not set")
wandb.login(key=WNB_API_KEY)

!wandb status

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/maso/.netrc
wandb: Currently logged in as: mosmar99 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Current Settings
{
  "_extra_http_headers": null,
  "_proxies": null,
  "api_key": null,
  "base_url": "https://api.wandb.ai",
  "entity": null,
  "git_remote": "origin",
  "ignore_globs": [],
  "organization": null,
  "project": null,
  "root_dir": null,
  "section": "default"
}


In [3]:
import gc
import torch
import weakref

def cleanup_gpu_memory(obj=None, verbose: bool = False):

    if not torch.cuda.is_available():
        if verbose:
            print("[INFO] CUDA is not available. No GPU cleanup needed.")
        return

    def get_memory_stats():
        allocated = torch.cuda.memory_allocated()
        reserved = torch.cuda.memory_reserved()
        return allocated, reserved

    torch.cuda.synchronize()

    if verbose:
        alloc, reserv = get_memory_stats()
        print(f"[Before] Allocated: {alloc / 1024**2:.2f} MB | Reserved: {reserv / 1024**2:.2f} MB")

    # Ensure we drop all strong references
    if obj is not None:
        ref = weakref.ref(obj)
        del obj
        if ref() is not None and verbose:
            print("[WARNING] Object not fully garbage collected yet.")

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    torch.cuda.synchronize()

    if verbose:
        alloc, reserv = get_memory_stats()
        print(f"[After]  Allocated: {alloc / 1024**2:.2f} MB | Reserved: {reserv / 1024**2:.2f} MB")

cleanup_gpu_memory(None, verbose=True)

[Before] Allocated: 0.00 MB | Reserved: 0.00 MB
[After]  Allocated: 0.00 MB | Reserved: 0.00 MB


In [5]:
import wandb
from ultralytics import YOLO

MODEL_NAME="yolo12m"

run = wandb.init(
    project="basketball_tracker",
    entity="baas_group5",
    job_type=f"finetune_{MODEL_NAME}"
)

model = YOLO(f"PATH_TO_WEIGHTS/{MODEL_NAME}.pt")

# --- CUSTOM W&B LOGGER ---
def extract_metrics(metrics):
    """Normalize YOLO metrics into a dict."""
    if metrics is None:
        return {}

    # If it's already a dict
    if isinstance(metrics, dict):
        return metrics

    # If it has a results_dict attribute that is a dict
    if hasattr(metrics, "results_dict") and isinstance(metrics.results_dict, dict):
        return metrics.results_dict

    # If results_dict is a method (older versions)
    if hasattr(metrics, "results_dict") and callable(metrics.results_dict):
        return metrics.results_dict()

    return {}

def on_train_epoch_end(trainer):
    m = extract_metrics(trainer.metrics)
    if m:
        wandb.log(m)

def on_val_end(trainer):
    m = extract_metrics(trainer.metrics)
    if m:
        wandb.log({f"val/{k}": v for k, v in m.items()})

model.add_callback("on_train_epoch_end", on_train_epoch_end)
model.add_callback("on_val_end", on_val_end)


results = model.train(
    data="basketball-player-detection-3-10/data.yaml",
    epochs=50,
    imgsz=640,
    lr0=0.01,
    lrf=0.001,
    cos_lr=True, 
    dropout=0.05,
)

best_path = f"{results.save_dir}/weights/best.pt"

artifact = wandb.Artifact(f"{MODEL_NAME}-imgsz720-ball-finetuned", type="model")
artifact.add_file(best_path)
run.log_artifact(artifact)

run.finish()


New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 5090, 32607MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=basketball-player-detection-3-10/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.05, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=PATH_TO_WEIGHTS/yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=Fa

metrics/mAP50(B),▁▆▃▂▆▇▆▇▆▇▇█▇▇███▇▇█████████████████████
metrics/mAP50-95(B),▁▃▂▄▄▅▆▆▆▆▇▆▅▇▇▇▇▇▇▇▇▇▇██▇█▇████████████
metrics/precision(B),▁▆▆▃▆▇▆▇▇▇▇▇▆▇▇▇█▇▆▇█▇██▇▇█▇█▇██████████
metrics/recall(B),▁▆▃▃▅▆▆▇▇▇▇█▇▇▇▇█▇▆▇▇▇▇▇▇▇██████████████
val/box_loss,▁▇██▇▇▇▆▆▆▆▆▇▆▆▆▆▆▆▆▆▆▆▆▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅
val/cls_loss,▁▇▃█▄▂▅▂▇▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val/dfl_loss,▁▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val/fitness,▄▂▁▄▃▅▆▅▆▆▇▆▅▇▇▇▆▇▆█▇▇█▇▇▇██▇███████████
val/metrics/mAP50(B),▆▂▁▅▆▇▆▇▇▇▇▆▇█▇█▇▇▇█████████████████████
val/metrics/mAP50-95(B),▄▂▁▄▃▅▆▅▆▆▇▆▅▇▇▆▆▇█▇▇▇█▇▇█▇██▇██████████
+2,...


In [3]:
import wandb
import os
from ultralytics import YOLO
import cv2

# ---------------------------
# CONFIG
# ---------------------------

TEAM = "baas_group5"
PROJECT = "basketball_tracker"

MODEL_ARTIFACT = f"{TEAM}/{PROJECT}/yolo12m-imgsz720-ball-finetuned:v2"
VIDEO_ARTIFACT = f"{TEAM}/{PROJECT}/videos:v0"

VIDEOS_TO_RUN = [
    "jokic_longer.mp4",
    "jokic_short.mp4"
]

# ---------------------------
# START W&B RUN
# ---------------------------

run = wandb.init(entity=TEAM, project=PROJECT, job_type="full_inference_pipeline")

# ---------------------------
# DOWNLOAD MODEL ARTIFACT
# ---------------------------

model_artifact = run.use_artifact(MODEL_ARTIFACT, type="model")
model_dir = model_artifact.download()

model_path = os.path.join(model_dir, "best.pt")
assert os.path.exists(model_path), f"Model weights not found: {model_path}"

print(f"[INFO] Using model weights: {model_path}")

# ---------------------------
# DOWNLOAD VIDEO ARTIFACT
# ---------------------------

video_artifact = run.use_artifact(VIDEO_ARTIFACT, type="dataset")
video_dir = video_artifact.download()

print(f"[INFO] Videos downloaded to: {video_dir}")

# ---------------------------
# RUN YOLO INFERENCE
# ---------------------------

model = YOLO(model_path)
output_dirs = []

for vid in VIDEOS_TO_RUN:
    src = os.path.join(video_dir, vid)
    assert os.path.exists(src), f"Video not found: {src}"

    print(f"[INFO] Running inference on: {src}")

    results = model.predict(
        source=src,
        save=True,      # saves annotated video to runs/predict/
        conf=0.50
    )

    save_dir = results[0].save_dir
    output_dirs.append(save_dir)

    print(f"[INFO] Output saved in: {save_dir}")

# ---------------------------
# UPLOAD INFERENCE OUTPUT ARTIFACT
# ---------------------------

inf_artifact = wandb.Artifact(
    name="yolo12m-inference-results",
    type="inference_results",
    description="YOLO12m inference on Jokic videos"
)

for out_dir in output_dirs:
    inf_artifact.add_dir(out_dir)

run.log_artifact(inf_artifact)

# ---------------------------
# DONE
# ---------------------------

print("\n[INFO] Pipeline complete — inference results uploaded to W&B.")
run.finish()


wandb:   1 of 1 files downloaded.  


[INFO] Using model weights: /home/maso/learn/robo_bt_tutorial/artifacts/yolo12m-imgsz720-ball-finetuned:v2/best.pt


wandb: Downloading large artifact 'videos:v0', 119.46MB. 2 files...
wandb:   2 of 2 files downloaded.  
Done. 00:00:00.3 (402.8MB/s)


[INFO] Videos downloaded to: /home/maso/learn/robo_bt_tutorial/artifacts/videos:v0
[INFO] Running inference on: /home/maso/learn/robo_bt_tutorial/artifacts/videos:v0/jokic_longer.mp4

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1287) /home/maso/learn/robo_bt_tutorial/artifacts/videos:v0/jokic_longer.mp4: 416x736 (no detections), 20.2ms
video 1/1 (frame 2/1287) /home/maso/learn/robo_bt_tutorial/artifacts/videos:v0/jokic_longer.mp4: 416x736 (no detections), 17.7ms
video 1/1 (fram

wandb: Adding directory to artifact (/home/maso/learn/robo_bt_tutorial/runs/detect/predict3)... 

[INFO] Output saved in: /home/maso/learn/robo_bt_tutorial/runs/detect/predict3


Done. 0.6s
wandb: Adding directory to artifact (/home/maso/learn/robo_bt_tutorial/runs/detect/predict3)... Done. 0.6s



[INFO] Pipeline complete — inference results uploaded to W&B.


In [5]:
import os
from ultralytics import YOLO
import cv2

# ---------------------------
# CONFIG (EDIT THESE)
# ---------------------------

MODEL_PATH = "runs/detect/train6/weights/best.pt"        # local YOLO weights
VIDEO_PATH = "videos/jokic_longer.mp4"       # local input video
OUTPUT_DIR = "runs/preds"        # directory to save results

# ---------------------------
# VALIDATION
# ---------------------------

assert os.path.exists(MODEL_PATH), f"Model weights not found: {MODEL_PATH}"
assert os.path.exists(VIDEO_PATH), f"Video not found: {VIDEO_PATH}"

print(f"[INFO] Using model weights: {MODEL_PATH}")
print(f"[INFO] Running inference on: {VIDEO_PATH}")

# ---------------------------
# RUN YOLO INFERENCE
# ---------------------------

model = YOLO(MODEL_PATH)

results = model.predict(
    source=VIDEO_PATH,
    save=True,              # will create runs/predict/ by default
    conf=0.50
)

# YOLO automatically chooses runs/predict/… folder
save_dir = results[0].save_dir

print(f"[INFO] YOLO saved results to: {save_dir}")

# Optionally: move output into your preferred directory
if OUTPUT_DIR:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.system(f"cp -r {save_dir}/* {OUTPUT_DIR}/")
    print(f"[INFO] Output copied to: {OUTPUT_DIR}")


[INFO] Using model weights: runs/detect/train6/weights/best.pt
[INFO] Running inference on: videos/jokic_longer.mp4

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1287) /home/maso/learn/robo_bt_tutorial/videos/jokic_longer.mp4: 384x640 (no detections), 21.5ms
video 1/1 (frame 2/1287) /home/maso/learn/robo_bt_tutorial/videos/jokic_longer.mp4: 384x640 (no detections), 16.4ms
video 1/1 (frame 3/1287) /home/maso/learn/robo_bt_tutorial/videos/jokic_longer.mp4: 384x640 (no detections),

In [6]:
import os
from roboflow import Roboflow
rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])
project = rf.workspace("asas-annotations").project("ai-sports-analytics-system")
version = project.version(7)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to AI-Sports-Analytics-System-7 in yolov12:: 100%|██████████| 32984/32984 [00:03<00:00, 9764.69it/s] 


In [4]:
import wandb
from ultralytics import YOLO

MODEL_NAME="yolo12m"

run = wandb.init(
    project="basketball_tracker",
    entity="baas_group5",
    job_type=f"finetune_{MODEL_NAME}"
)

model = YOLO(f"PATH_TO_WEIGHTS/{MODEL_NAME}.pt")

# --- CUSTOM W&B LOGGER ---
def extract_metrics(metrics):
    """Normalize YOLO metrics into a dict."""
    if metrics is None:
        return {}

    # If it's already a dict
    if isinstance(metrics, dict):
        return metrics

    # If it has a results_dict attribute that is a dict
    if hasattr(metrics, "results_dict") and isinstance(metrics.results_dict, dict):
        return metrics.results_dict

    # If results_dict is a method (older versions)
    if hasattr(metrics, "results_dict") and callable(metrics.results_dict):
        return metrics.results_dict()

    return {}

def on_train_epoch_end(trainer):
    m = extract_metrics(trainer.metrics)
    if m:
        wandb.log(m)

def on_val_end(trainer):
    m = extract_metrics(trainer.metrics)
    if m:
        wandb.log({f"val/{k}": v for k, v in m.items()})

model.add_callback("on_train_epoch_end", on_train_epoch_end)
model.add_callback("on_val_end", on_val_end)


results = model.train(
    data="AI-Sports-Analytics-System-7/data.yaml",
    epochs=20,
    imgsz=640,
    lr0=0.01,
    lrf=0.001,
    cos_lr=True, 
    dropout=0.05,
    batch=24
)

best_path = f"{results.save_dir}/weights/best.pt"

artifact = wandb.Artifact(f"{MODEL_NAME}-imgsz720-ball-finetuned", type="model")
artifact.add_file(best_path)
run.log_artifact(artifact)

run.finish()

New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 5090, 32607MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=AI-Sports-Analytics-System-7/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.05, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=PATH_TO_WEIGHTS/yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False,

metrics/mAP50(B),▁▅▆▆▇▇█▇████████████
metrics/mAP50-95(B),▁▅▅▅▇▇▇▇▇▇▇▇█▇▇█████
metrics/precision(B),▁▇▇▇▇███████████████
metrics/recall(B),▁▅▆▆▇▇▇▇▇▇██████████
val/box_loss,▁▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▇▇
val/cls_loss,▁█▆▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
val/dfl_loss,▁█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val/fitness,▂▁▂▆▆▇▆▇▆▇▆▇▇▇███████
val/metrics/mAP50(B),▁▃▄▆▆▇▇▇▇▇▇██████████
val/metrics/mAP50-95(B),▂▁▂▆▆▇▆▇▆▇▆▇▇▇███████
+2,...
